##  Classifying audio data with convolutional neural networks

<br/>
by Eduardo Garcia Rajo<br/>
<br/>

This notebook is part of the project "Urban sounds classification with Covnolutional Neural Networks" on [my Github](https://github.com/GorillaBus/urban-audio-classifier).<br/>
<br/>
Licensed under the GNU LESSER GENERAL PUBLIC LICENSE Version 3, 29 June 2007<br/>
<br/>

# UrbanSound8K Data Augmentation

On this notebook I'm creating variations of the original files applying the following effects:

1. Time stretch
2. Pitch shift
3. Noise

By default the new files are being saved in **UrbanSound8K/audio/augmented** with the same fold schema. At the end of the notebook a new metadata file is created combining the original dataset files and the generated ones.<br/>
<br/>
<br/>

In [36]:
import os
import librosa
import librosa.display
import pandas as pd
import numpy as np
import random
import matplotlib as plt
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

#### * Load metadata CSV

In [37]:
# Define general variables

# Set your path to the dataset
us8k_path = os.path.abspath('./data/UrbanSound8K')
audio_path = os.path.join(us8k_path, 'audio')
augmented_path = os.path.join(audio_path, 'augmented')

# Metadata
metadata_path = os.path.join(us8k_path, 'metadata/UrbanSound8K.csv')
metadata_augmented_path = os.path.abspath('data/augmented-data.csv')

#### * Load metadata

In [38]:
# Load the metadata from the generated CSV
metadata = pd.read_csv(metadata_path)

# Examine dataframe
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


#### 1. Time stretching (changing play time)

In [39]:
import soundfile
rates = [0.81, 1.07]
total = len(metadata) * len(rates)
count = 0
for rate in rates: 
    # Generate new stretched audio file
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']
        
        # Speed sub-dir inside current fold dir
        curr_rate_path = augmented_path + '/fold' + curr_fold + '/speed_' + str(int(rate*100))

        
        # Create sub-dir if it does not exist
        if not os.path.exists(curr_rate_path):
            os.makedirs(curr_rate_path)
                    
        output_path = curr_rate_path + '/' + row['slice_file_name']
        
        # Skip when file already exists
        if (os.path.isfile(output_path)):
            count += 1 
            continue
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.time_stretch(y, rate=rate)
        soundfile.write(output_path, y_changed, sr)
        
        count += 1 
        
        clear_output(wait=True)
        print("Progress: {}/{}".format(count, total))
        print("Last file: ", row['slice_file_name'])


#### 2. Pitch shifting

In [40]:
tone_steps = [-1, -2, 1, 2]
total = len(metadata) * len(tone_steps)
count = 0
for tone_step in tone_steps:
    # Generate new pitched audio
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

        # Pitch Shift sub-dir inside current fold dir
        curr_ps_path = augmented_path + '/fold' + curr_fold + '/pitch_' + str(tone_step)

        # Create sub-dir if it does not exist
        if not os.path.exists(curr_ps_path):
            os.makedirs(curr_ps_path)
        
        output_path = curr_ps_path + '/' + row['slice_file_name']
        
        # Skip when file already exists
        if (os.path.isfile(output_path)):
            count += 1 
            continue
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.pitch_shift(y, sr, n_steps=tone_step)
        soundfile.write(output_path, y_changed, sr)
        
        count += 1 
        
        clear_output(wait=True)
        print("Progress: {}/{}".format(count, total))
        print("Last file: ", row['slice_file_name'])

#### 3. Noise

In [41]:
import random

def add_noise(data):
    noise = np.random.rand(len(data))
    noise_amp = random.uniform(0.005, 0.008)
    data_noise = data + (noise_amp * noise)
    return data_noise

total = len(metadata)
count = 0

# Generate new noised audio
for index, row in metadata.iterrows():        
    curr_fold = str(row['fold'])
    curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

    # Noised sub-dir inside current fold dir
    curr_noise_path = augmented_path + '/fold' + curr_fold + '/noise'

    # Create sub-dir if it does not exist
    if not os.path.exists(curr_noise_path):
        os.makedirs(curr_noise_path)
        
    output_path = curr_noise_path + '/' + row['slice_file_name']
        
    # Skip when file already exists
    if (os.path.isfile(output_path)):
        count += 1 
        continue
        
    y, sr = librosa.load(curr_file_path)  
    y_changed = add_noise(y)
    soundfile.write(output_path, y_changed, sr)
    
    count += 1 

    clear_output(wait=True)
    print("Progress: {}/{}".format(count, total))
    print("Last file: ", row['slice_file_name'])

#### 4. Create metadata for the new files

In [42]:
def get_files_recursive(path):
    # create a list of file and sub directories names in the given directory 
    file_list = os.listdir(path)
    all_files = list()
    # Iterate over all the entries
    for entry in file_list:
        # Create full path
        full_path = os.path.join(path, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(full_path):
            all_files = all_files + get_files_recursive(full_path)
        else:
            all_files.append(full_path)
                
    return all_files   

In [30]:
files = get_files_recursive(augmented_path)
print(files[0])

C:\dev\volgemobile\train\data\UrbanSound8K\audio\augmented\fold1\noise\101415-3-0-2.wav


In [51]:
# Get every single file within the tree
files = get_files_recursive(augmented_path)

# Define metadata columns
names = []
classes = []
folds = []
augmentations = []

# Recursively split the pieces
def get_file_pieces(file_path,pieces=None):
    if pieces is None:
        pieces=[]

    new_pieces = os.path.split(file_path)
    if new_pieces[1]=="":
        pieces.append(new_pieces[0])
        return pieces
    pieces.append(new_pieces[1])

    return get_file_pieces(new_pieces[0],pieces)
        
# Iterate and collect name, fold and class
for file in files:
    pieces = get_file_pieces(file)
    pieces.reverse()
    file = pieces[len(pieces) - 1]
    fold = pieces[len(pieces) - 3] 
    augment = pieces[len(pieces) - 2] 
    fold_num = fold[4:len(fold)]
    file_name_pieces = pieces[-1].split("-")
    if len(file_name_pieces)<2:
        print(file)
        print(file_name_pieces)
        print("File name is too short")
        continue
    print(file)
    class_id = file_name_pieces[1]
    
    # Push records
    names.append(file)
    folds.append(fold_num)
    classes.append(class_id)
    augmentations.append(augment)

# Create a dataframe with the new augmented data
new_meta = pd.DataFrame({'file': names, 'fold': folds, 'class_id': classes, 'augment': augmentations })

# Make sure class_id is int
new_meta['class_id'] = new_meta['class_id'].astype(np.int64)

print(len(new_meta), "new entries")

101415-3-0-2.wav
101415-3-0-3.wav
101415-3-0-8.wav
102106-3-0-0.wav
102305-6-0-0.wav
102842-3-0-1.wav
102842-3-1-0.wav
102842-3-1-5.wav
102842-3-1-6.wav
103074-7-0-0.wav
103074-7-0-1.wav
103074-7-0-2.wav
103074-7-1-0.wav
103074-7-1-1.wav
103074-7-1-2.wav
103074-7-1-3.wav
103074-7-1-4.wav
103074-7-1-5.wav
103074-7-1-6.wav
103074-7-2-0.wav
103074-7-3-0.wav
103074-7-3-1.wav
103074-7-3-2.wav
103074-7-3-3.wav
103074-7-4-0.wav
103074-7-4-1.wav
103074-7-4-2.wav
103074-7-4-3.wav
103074-7-4-4.wav
103074-7-4-5.wav
103074-7-4-6.wav
103258-5-0-0.wav
103258-5-0-1.wav
103258-5-0-10.wav
103258-5-0-11.wav
103258-5-0-12.wav
103258-5-0-13.wav
103258-5-0-14.wav
103258-5-0-15.wav
103258-5-0-16.wav
103258-5-0-17.wav
103258-5-0-18.wav
103258-5-0-19.wav
103258-5-0-2.wav
103258-5-0-3.wav
103258-5-0-4.wav
103258-5-0-5.wav
103258-5-0-6.wav
103258-5-0-7.wav
103258-5-0-8.wav
103258-5-0-9.wav
105415-2-0-1.wav
105415-2-0-15.wav
105415-2-0-19.wav
105415-2-0-21.wav
105415-2-0-24.wav
105415-2-0-4.wav
105415-2-0-6.wav


159738-8-0-10.wav
159738-8-0-11.wav
159738-8-0-12.wav
159738-8-0-13.wav
159738-8-0-14.wav
159738-8-0-15.wav
159738-8-0-16.wav
159738-8-0-17.wav
159738-8-0-18.wav
159738-8-0-19.wav
159738-8-0-2.wav
159738-8-0-3.wav
159738-8-0-4.wav
159738-8-0-5.wav
159738-8-0-6.wav
159738-8-0-7.wav
159738-8-0-8.wav
159738-8-0-9.wav
160009-2-0-1.wav
160009-2-0-11.wav
160009-2-0-22.wav
160009-2-0-30.wav
160009-2-0-31.wav
160009-2-0-47.wav
160009-2-0-5.wav
160009-2-0-50.wav
160094-3-0-0.wav
161702-9-0-26.wav
161702-9-0-30.wav
161702-9-0-37.wav
161702-9-0-45.wav
161702-9-0-6.wav
161702-9-0-8.wav
162540-1-0-0.wav
162540-1-1-0.wav
162540-1-2-0.wav
162703-1-0-0.wav
164053-8-0-0.wav
164053-8-1-0.wav
164053-8-2-0.wav
164053-8-2-1.wav
164053-8-2-2.wav
164053-8-2-3.wav
164053-8-3-0.wav
164312-3-0-0.wav
164312-3-1-0.wav
164312-3-3-0.wav
164312-3-4-0.wav
165067-2-0-111.wav
165067-2-0-112.wav
165067-2-0-35.wav
165067-2-0-56.wav
165067-2-0-72.wav
165067-2-0-9.wav
165067-2-0-91.wav
165642-4-0-0.wav
165645-4-0-0.wav
165

40722-8-0-7.wav
40722-8-0-8.wav
43786-3-0-0.wav
43786-3-1-0.wav
43806-8-0-0.wav
44736-5-0-0.wav
44736-5-0-1.wav
46654-6-0-0.wav
46656-6-0-0.wav
46656-6-1-0.wav
46656-6-2-0.wav
46656-6-3-0.wav
46656-6-4-0.wav
46656-6-5-0.wav
46669-4-0-0.wav
46669-4-0-10.wav
46669-4-0-2.wav
46669-4-0-24.wav
46669-4-0-26.wav
46669-4-0-3.wav
46669-4-0-30.wav
46669-4-0-31.wav
46669-4-0-32.wav
46669-4-0-33.wav
46669-4-0-35.wav
46669-4-0-36.wav
46669-4-0-37.wav
46669-4-0-39.wav
46669-4-0-4.wav
46669-4-0-45.wav
46669-4-0-48.wav
46669-4-0-49.wav
46669-4-0-54.wav
46669-4-0-61.wav
46669-4-0-63.wav
46669-4-0-8.wav
46669-4-0-9.wav
46918-5-0-0.wav
46918-5-0-1.wav
46918-5-0-2.wav
50901-0-0-0.wav
50901-0-1-0.wav
50901-0-1-1.wav
51022-3-13-0.wav
51022-3-29-0.wav
51022-3-30-4.wav
51022-3-7-0.wav
54858-3-0-0.wav
54858-3-1-2.wav
54858-3-2-0.wav
55020-4-0-0.wav
55020-4-0-1.wav
55020-4-0-10.wav
55020-4-0-11.wav
55020-4-0-12.wav
55020-4-0-13.wav
55020-4-0-14.wav
55020-4-0-2.wav
55020-4-0-3.wav
55020-4-0-4.wav
55020-4-0-5.wav

51024-3-0-23.wav
51024-3-0-3.wav
54086-1-0-0.wav
54086-1-1-0.wav
54086-1-2-0.wav
59513-3-0-0.wav
59513-3-0-1.wav
59513-3-0-3.wav
59513-3-0-4.wav
66115-1-0-0.wav
66115-1-0-1.wav
69661-3-0-14.wav
69661-3-0-23.wav
69661-3-0-29.wav
7062-6-0-0.wav
71312-3-0-1.wav
71312-3-0-2.wav
71312-3-0-4.wav
71312-3-0-5.wav
72220-3-0-0.wav
72220-3-2-2.wav
72220-3-2-3.wav
72220-3-2-4.wav
72538-3-0-0.wav
72538-3-0-1.wav
72538-3-0-3.wav
72538-3-0-4.wav
73524-0-0-0.wav
73524-0-0-101.wav
73524-0-0-105.wav
73524-0-0-113.wav
73524-0-0-114.wav
73524-0-0-126.wav
73524-0-0-127.wav
73524-0-0-128.wav
73524-0-0-133.wav
73524-0-0-137.wav
73524-0-0-2.wav
73524-0-0-23.wav
73524-0-0-30.wav
73524-0-0-31.wav
73524-0-0-32.wav
73524-0-0-5.wav
73524-0-0-52.wav
73524-0-0-54.wav
73524-0-0-6.wav
73524-0-0-63.wav
73524-0-0-73.wav
73524-0-0-74.wav
73524-0-0-8.wav
73524-0-0-92.wav
73524-0-0-98.wav
73524-0-0-99.wav
74364-8-1-0.wav
74364-8-1-1.wav
74364-8-1-10.wav
74364-8-1-11.wav
74364-8-1-12.wav
74364-8-1-13.wav
74364-8-1-14.wav
74

83195-9-0-11.wav
83195-9-0-14.wav
83195-9-0-2.wav
83195-9-0-3.wav
83195-9-0-6.wav
83195-9-0-7.wav
83502-0-0-0.wav
83502-0-0-1.wav
83502-0-0-10.wav
83502-0-0-11.wav
83502-0-0-12.wav
83502-0-0-13.wav
83502-0-0-2.wav
83502-0-0-3.wav
83502-0-0-4.wav
83502-0-0-5.wav
83502-0-0-6.wav
83502-0-0-7.wav
83502-0-0-8.wav
83502-0-0-9.wav
84249-9-0-10.wav
84249-9-0-11.wav
84249-9-0-3.wav
84249-9-0-6.wav
84249-9-0-7.wav
84249-9-0-8.wav
85574-3-0-0.wav
85574-3-0-12.wav
85574-3-0-2.wav
85574-3-0-8.wav
85665-3-0-0.wav
88121-8-0-0.wav
88121-8-0-1.wav
88121-8-0-2.wav
88121-8-0-3.wav
88121-8-1-0.wav
88121-8-2-0.wav
93567-8-0-0.wav
93567-8-0-1.wav
93567-8-0-10.wav
93567-8-0-11.wav
93567-8-0-12.wav
93567-8-0-13.wav
93567-8-0-14.wav
93567-8-0-15.wav
93567-8-0-16.wav
93567-8-0-17.wav
93567-8-0-18.wav
93567-8-0-19.wav
93567-8-0-2.wav
93567-8-0-3.wav
93567-8-0-4.wav
93567-8-0-5.wav
93567-8-0-6.wav
93567-8-0-7.wav
93567-8-0-8.wav
93567-8-0-9.wav
97756-3-0-0.wav
99192-4-0-0.wav
99192-4-0-1.wav
99192-4-0-12.wav
9919

...

125678-7-3-1.wav
125678-7-3-2.wav
125678-7-3-3.wav
125678-7-3-4.wav
125678-7-3-5.wav
125678-7-3-6.wav
125678-7-3-7.wav
125678-7-4-0.wav
125678-7-4-1.wav
125678-7-4-2.wav
125678-7-4-3.wav
125678-7-4-4.wav
125678-7-4-5.wav
125678-7-4-6.wav
125678-7-4-7.wav
125678-7-4-8.wav
126153-9-0-0.wav
126153-9-0-1.wav
126153-9-0-11.wav
126153-9-0-4.wav
126153-9-0-5.wav
126153-9-0-6.wav
126153-9-0-8.wav
131571-6-0-0.wav
133090-2-0-14.wav
133090-2-0-20.wav
133090-2-0-28.wav
133090-2-0-37.wav
133090-2-0-38.wav
133090-2-0-64.wav
133090-2-0-70.wav
133090-2-0-76.wav
133473-8-0-0.wav
133473-8-0-1.wav
133473-8-0-2.wav
133473-8-0-3.wav
133473-8-0-4.wav
133473-8-0-5.wav
138465-1-0-0.wav
139000-4-0-0.wav
139000-4-0-1.wav
139000-4-0-2.wav
142003-2-0-0.wav
142003-2-0-1.wav
142003-2-0-19.wav
142003-2-0-20.wav
142003-2-0-21.wav
142003-2-0-24.wav
142003-2-0-30.wav
142003-2-0-8.wav
14386-9-0-11.wav
14386-9-0-16.wav
14386-9-0-17.wav
14386-9-0-2.wav
14386-9-0-20.wav
14386-9-0-6.wav
144028-3-0-0.wav
144028-3-1-0.wav
14

14386-9-0-2.wav
14386-9-0-20.wav
14386-9-0-6.wav
144028-3-0-0.wav
144028-3-1-0.wav
144028-3-1-1.wav
145206-6-0-0.wav
145206-6-1-0.wav
145206-6-2-0.wav
145206-6-3-0.wav
145206-6-4-0.wav
145206-6-5-0.wav
145612-6-0-0.wav
145612-6-1-0.wav
145612-6-2-0.wav
145612-6-3-0.wav
146343-3-0-0.wav
147019-4-0-0.wav
148632-8-0-0.wav
148632-8-0-1.wav
148632-8-0-10.wav
148632-8-0-11.wav
148632-8-0-12.wav
148632-8-0-13.wav
148632-8-0-14.wav
148632-8-0-15.wav
148632-8-0-16.wav
148632-8-0-17.wav
148632-8-0-18.wav
148632-8-0-2.wav
148632-8-0-3.wav
148632-8-0-4.wav
148632-8-0-5.wav
148632-8-0-6.wav
148632-8-0-7.wav
148632-8-0-8.wav
148632-8-0-9.wav
151069-6-0-0.wav
155217-9-0-27.wav
155217-9-0-39.wav
155217-9-0-57.wav
155217-9-0-85.wav
155217-9-1-24.wav
155217-9-1-58.wav
155283-1-0-0.wav
155283-1-1-0.wav
155294-3-0-0.wav
155313-3-0-0.wav
156358-5-0-0.wav
156358-5-0-1.wav
156358-5-0-2.wav
156358-5-0-3.wav
156358-5-0-4.wav
156358-5-0-5.wav
156358-5-0-6.wav
157649-3-0-0.wav
157649-3-0-1.wav
157649-3-1-1.wav
1

145390-9-0-15.wav
145390-9-0-22.wav
145390-9-0-26.wav
145390-9-0-3.wav
145390-9-0-34.wav
145390-9-0-7.wav
145683-6-0-0.wav
145683-6-1-0.wav
145683-6-2-0.wav
145683-6-3-0.wav
145683-6-4-0.wav
145683-6-5-0.wav
146249-4-0-0.wav
148166-6-0-0.wav
149255-9-0-0.wav
149255-9-0-13.wav
149255-9-0-3.wav
149255-9-0-4.wav
149255-9-0-5.wav
149255-9-0-7.wav
149929-9-0-0.wav
149929-9-0-2.wav
149929-9-0-5.wav
149929-9-1-0.wav
149929-9-1-1.wav
149929-9-1-4.wav
152570-9-0-0.wav
152570-9-1-41.wav
152570-9-1-42.wav
152570-9-1-61.wav
152570-9-1-63.wav
152570-9-1-81.wav
152588-3-0-0.wav
152588-3-1-0.wav
152588-3-2-0.wav
152588-3-3-0.wav
155044-9-0-11.wav
155044-9-0-15.wav
155044-9-0-19.wav
155044-9-0-3.wav
155044-9-0-37.wav
155044-9-0-38.wav
155130-1-0-0.wav
155130-1-1-0.wav
155242-9-0-12.wav
155242-9-0-15.wav
155242-9-0-27.wav
155242-9-0-35.wav
155242-9-0-4.wav
155242-9-0-44.wav
155312-3-0-0.wav
156200-2-0-0.wav
157801-6-0-0.wav
157866-8-0-0.wav
157866-8-0-1.wav
157866-8-0-10.wav
157866-8-0-11.wav
157866-8-

39856-5-0-19.wav
39856-5-0-2.wav
39856-5-0-20.wav
39856-5-0-21.wav
39856-5-0-22.wav
39856-5-0-23.wav
39856-5-0-24.wav
39856-5-0-25.wav
39856-5-0-26.wav
39856-5-0-27.wav
39856-5-0-28.wav
39856-5-0-3.wav
39856-5-0-4.wav
39856-5-0-5.wav
39856-5-0-6.wav
39856-5-0-7.wav
39856-5-0-8.wav
39856-5-0-9.wav
42101-1-1-0.wav
42371-3-0-0.wav
42953-9-0-16.wav
42953-9-0-21.wav
42953-9-0-28.wav
42953-9-0-37.wav
42953-9-0-49.wav
42953-9-0-50.wav
50223-3-0-1.wav
50223-3-0-10.wav
50223-3-0-3.wav
50223-3-0-6.wav
50414-4-0-0.wav
50414-4-0-1.wav
52171-3-2-0.wav
52171-3-3-0.wav
52171-3-6-0.wav
52171-3-6-1.wav
52411-9-0-4.wav
52411-9-0-52.wav
52411-9-0-6.wav
52411-9-0-64.wav
52411-9-0-66.wav
52411-9-0-79.wav
52740-3-0-0.wav
52740-3-0-1.wav
52740-3-0-2.wav
52740-3-0-3.wav
54187-1-0-0.wav
54187-1-0-1.wav
54187-1-0-2.wav
54187-1-0-3.wav
54187-1-0-4.wav
54823-3-0-0.wav
54823-3-1-0.wav
54823-3-2-1.wav
54976-4-0-0.wav
54976-4-0-1.wav
54976-4-0-2.wav
54976-4-0-3.wav
54976-4-0-4.wav
54976-4-0-5.wav
54976-4-0-6.wav
549

61124 new entries


In [52]:
# Add class names to the new dataframe using merge
classes = pd.DataFrame({
    'class_id': range(0,10),
    'class': [
        'air_conditioner',
        'car_horn',
        'children_playing',
        'dog_bark',
        'drilling',
        'engine_idling',
        'gun_shot',
        'jackhammer',
        'siren',
        'street_music'
    ]
})

new_meta = pd.merge(new_meta, classes, on='class_id')


In [53]:
new_meta.tail()

,file,fold,class_id,augment,class
61119,54187-1-0-3.wav,9,1,speed_81,car_horn
61120,54187-1-0-4.wav,9,1,speed_81,car_horn
61121,73168-1-0-0.wav,9,1,speed_81,car_horn
61122,73168-1-1-0.wav,9,1,speed_81,car_horn
61123,73168-1-2-0.wav,9,1,speed_81,car_horn


#### 5. Integrate metadata in a single file

In [58]:
# Modify original data to fit the new structure
try:
    del metadata['fsID']
except:
    print("Oops")
try:
    del metadata['start']
except:
    print("Oops")
try:
    del metadata['end']
except:
    print("Oops")
try:
    del metadata['salience']
except:
    print("Oops")
print(metadata.head)
print(list(metadata.columns.values))
metadata.columns = ['file', 'fold', 'class_id', 'class']
metadata['augment'] = 'none'



Oops
Oops
Oops
Oops
<bound method NDFrame.head of                     file  fold  class_id             class augment
0       100032-3-0-0.wav     5         3          dog_bark    none
1     100263-2-0-117.wav     5         2  children_playing    none
2     100263-2-0-121.wav     5         2  children_playing    none
3     100263-2-0-126.wav     5         2  children_playing    none
4     100263-2-0-137.wav     5         2  children_playing    none
...                  ...   ...       ...               ...     ...
8727     99812-1-2-0.wav     7         1          car_horn    none
8728     99812-1-3-0.wav     7         1          car_horn    none
8729     99812-1-4-0.wav     7         1          car_horn    none
8730     99812-1-5-0.wav     7         1          car_horn    none
8731     99812-1-6-0.wav     7         1          car_horn    none

[8732 rows x 5 columns]>
['file', 'fold', 'class_id', 'class', 'augment']


ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [59]:
# Concat the two dataframes
full_meta = pd.concat([metadata, new_meta])

# Verify lengths
if (len(full_meta) == len(metadata) + len(new_meta)):
    print("Dataframes merged correctly!")
else:
    print("Error! Lengths do not match.")

print("Initial data:", len(metadata))
print("New data:", len(new_meta))
print("Merged data:", len(full_meta))

Dataframes merged correctly!
Initial data: 8732
New data: 61124
Merged data: 69856


####  6. Save the new dataset

In [60]:
# Save the new metadata
full_meta.to_csv(metadata_augmented_path, index=False, encoding="utf-8")